In [1]:
%load_ext autoreload
%autoreload 2
#%reload_ext autoreload

In [1]:
import tensorflow as tf
import tensorflow.keras.layers as layers
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import Generator
import model_definitions

### Open input file

In [2]:
SEQUENCE_LENGTH=20
ONE_CAR=True
PATH=r"D:\Users\Students\Daniele\records\OneCar_db.txt"
BATCH_SIZE=256*2
FEATURES=7 
FEATURES_PREDICTED=3
DISCARD=19

In [3]:
maxSpeed=40
frequency=0.02
maxdiff=maxSpeed*frequency
maxAngularVelocity=140 #140 degrees for second
maxAngDiff=maxAngularVelocity*frequency
print(f"Max speed diff: {maxdiff}\nMax x/z diff: {maxdiff}\nMax rot diff: {maxAngDiff}")

Max speed diff: 0.8
Max x/z diff: 0.8
Max rot diff: 2.8000000000000003


In [4]:
Generator.SEQUENCE_LENGTH=SEQUENCE_LENGTH
Generator.FEATURES=FEATURES
Generator.DISCARD=DISCARD
df=Generator.single_care_dataframe(PATH)

In [5]:
def SubtractDF(df):
    return df.groupby("RACE",group_keys=False).apply(Generator.subtraction_columns)

dfs=[]
for i in range(len(df)):
    dfs.append(SubtractDF(df[i]))
    #dfs.append(df[i])

united=dfs[0]
for i in range(1,len(df)):
    united=pd.concat([united,dfs[i] ],ignore_index=True)

In [6]:
united.tail()

,X,Z,VEL_X,VEL_Z,ROT,ANG_VEL_Y,ACC_X,ACC_Z,TILE,TILE_IND,X_RELATIVE,Z_RELATIVE,TIME,RACE,GROUP
5310813,7.8144,9.0694,1.96719,-1.65401,4.59076,0.013007,15.948295,-4.203892,8,16,0.133450,-0.146814,0.4,167,19
5310814,8.8192,8.4603,2.37350,-1.55413,5.25821,0.040471,-0.228595,-0.795459,8,16,0.192244,-0.090412,0.4,167,19
5310815,9.7417,7.6619,1.91026,-1.37267,3.26049,-0.154846,-3.500080,3.886413,8,16,0.257189,-0.039332,0.4,167,19
5310816,10.2719,7.5774,0.80994,0.08210,0.62741,0.050259,-2.398968,-0.017929,8,16,0.325669,0.011184,0.4,167,19
5310817,9.5723,7.7028,-5.61399,1.94441,-11.89370,-0.690471,-21.933367,8.758164,8,16,0.389484,0.062536,0.4,167,19


In [7]:
df_train,df_val,df_test= Generator.split_train_validation_test(united,["RACE","GROUP"])

In [8]:
df_train.head()

,X,Z,VEL_X,VEL_Z,ROT,ANG_VEL_Y,ACC_X,ACC_Z,TILE,TILE_IND,X_RELATIVE,Z_RELATIVE,TIME,RACE,GROUP
0,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0,0,0.000000,0.000000,0.0,0,0
1,-0.20391,1.377150,-0.402967,5.795482,-5.7704,-0.142928,33.748100,2.395153,4,32,-0.126719,0.407695,0.4,0,0
2,-0.47607,2.941160,-0.788368,3.021808,-1.9582,0.148060,-35.617326,7.960847,4,32,-0.133066,0.446911,0.4,0,0
3,-0.47730,4.308594,-0.153198,3.299600,1.6448,-0.058761,-3.675389,-11.252597,16,0,-0.139430,-0.491283,0.4,0,0
4,-0.69836,5.443848,-0.555344,2.914620,-1.0694,0.069873,4.957903,7.639074,16,0,-0.148742,-0.346113,0.4,0,0


In [12]:
gen_train=Generator.DataGenerator(BATCH_SIZE,df_train,5000)
gen_val=Generator.DataGenerator(BATCH_SIZE,df_val,2400)
gen_test=Generator.DataGenerator(BATCH_SIZE,df_test,2400)


Length: 2654563 races: 168 n batches: 2641123 / 512
Length: 1327368 races: 168 n batches: 1313928 / 512
Length: 1328887 races: 168 n batches: 1315447 / 512


In [22]:
#x_y=gen_test[0]
#x=x_y[0][0]
#print(x)
#x_y[1][0]
1313928/512

2566.265625

### Model definition

#### Train and print

In [13]:
if(ONE_CAR):
    model_definitions.cars=1
else:
    model_definitions.cars=4
model_definitions.features=FEATURES
model_definitions.sequence_length=SEQUENCE_LENGTH
model_definitions.feat_pred=FEATURES_PREDICTED

### Rules Model

In [ ]:
lstm,history=LSTM_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.01,epoch=100, dropout=0.2, cells=6 )

ev=lstm.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
lstm2,history=LSTM_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.002,epoch=100, dropout=0.02, cells=1)

ev=lstm2.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
mlp,history=MLP_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.001,epoch=100)

ev=mlp.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

In [ ]:
siamese,history=LSTM_Siamese_Train(2,SEQUENCE_LENGTH,4,rules_train,rules_val,learning_rate=0.001,epoch=100, dropout=0.02, cells=2 )

ev=siamese.evaluate(rules_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

### Physichs Model

Since physichs data can be very small, Each data is multiply by 10 

In [71]:
NAME=f"MLP_DISCARD_{DISCARD}_DELTA_LONG_TRAIN"
mlp_ph,history_mlp_ph=model_definitions.MLP_Train(gen_train,gen_val,learning_rate=0.0001,epoch=1000,verbose=1,name=NAME,path="logs")

ev=mlp_ph.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")
tf.saved_model.save(mlp_ph,f'saved_model/{NAME}')

Epoch 1/1000
5109/5109 [==============================] - 497s 97ms/step - loss: 6.9352 - mean_absolute_error: 1.6829 - accuracy: 0.0000e+00 - val_loss: 5.1149 - val_mean_absolute_error: 1.4049 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 2/1000
5109/5109 [==============================] - 499s 98ms/step - loss: 4.8177 - mean_absolute_error: 1.3599 - accuracy: 0.0000e+00 - val_loss: 5.0952 - val_mean_absolute_error: 1.3989 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 3/1000
5109/5109 [==============================] - 497s 97ms/step - loss: 4.8111 - mean_absolute_error: 1.3584 - accuracy: 0.0000e+00 - val_loss: 5.0931 - val_mean_absolute_error: 1.3986 - val_accuracy: 0.0000e+00 - lr: 1.0000e-04
Epoch 4/1000
5108/5109 [============================>.] - ETA: 0s - loss: 4.8108 - mean_absolute_error: 1.3583 - accuracy: 1.2746e-07
Epoch 4: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.
5109/5109 [==============================] - 496s 97ms/step - loss: 4.8111 -

In [ ]:
units=32
kernel_size=1
filters=32
normalization=False
cnn_lstm,history_cnn_lstm=model_definitions.CNN_LSTM_Train(gen_train,gen_val,learning_rate=0.0001,patience=10,name=f"CNN{filters}_{kernel_size}LSTM{units}_Norm{normalization}",
                                                           path="logs_350SequenceLength",epoch=10, dropout=0.2,units=units,kernel_size=kernel_size,
                                                           filters=filters, normalization=normalization,verbose=1
                                                          )

ev=cnn_lstm.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")

tf.saved_model.save(cnn_lstm,f'saved_model/CNN_LSTM{units}_{kernel_size}_{filters}_PosAssoluta')

In [14]:
lr =1e-3
units=512
cells=8
normalization=False
name=f"LSTM{cells}_{units}_{lr}_DISCARD_{DISCARD}_DELTA"
lstm_ph,history_lstm_ph=model_definitions.LSTM_Train(gen_train,gen_val,learning_rate=lr,patience=50,name=name,path="logs",
                                                     epoch=100, dropout=0.2, cells=cells,units=units, normalization=normalization,verbose=1)

ev=lstm_ph.evaluate(gen_test,verbose=0)
print(f"Loss: {ev[0]} MAE: {ev[1]} Accuracy: {ev[2]}")
tf.saved_model.save(lstm_ph,f'saved_model/{name}')

Epoch 1/100
5000/5000 [==============================] - 610s 120ms/step - loss: 5.4493 - mean_absolute_error: 1.4072 - accuracy: 2.6042e-07 - val_loss: 13.9756 - val_mean_absolute_error: 2.3651 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 2/100
5000/5000 [==============================] - 600s 120ms/step - loss: 3.9807 - mean_absolute_error: 1.2171 - accuracy: 2.6042e-07 - val_loss: 12.9562 - val_mean_absolute_error: 2.2731 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 3/100
5000/5000 [==============================] - 602s 120ms/step - loss: 3.7403 - mean_absolute_error: 1.1773 - accuracy: 2.6042e-07 - val_loss: 11.3177 - val_mean_absolute_error: 2.1094 - val_accuracy: 0.0000e+00 - lr: 0.0010
Epoch 4/100
5000/5000 [==============================] - 600s 120ms/step - loss: 3.6258 - mean_absolute_error: 1.1565 - accuracy: 0.0000e+00 - val_loss: 10.6564 - val_mean_absolute_error: 2.0394 - val_accuracy: 2.7127e-07 - lr: 0.0010
Epoch 5/100
5000/5000 [==============================] - 602

INFO:tensorflow:Assets written to: saved_model/LSTM8_512_0.001_DISCARD_19_DELTA\assets


INFO:tensorflow:Assets written to: saved_model/LSTM8_512_0.001_DISCARD_19_DELTA\assets


In [29]:
tf.saved_model.save(lstm_ph,f'saved_model/{name}')

INFO:tensorflow:Assets written to: saved_model/LSTM4_128_0.001_DISCARD_19_DELTA\assets


INFO:tensorflow:Assets written to: saved_model/LSTM4_128_0.001_DISCARD_19_DELTA\assets


In [19]:
model=model_definitions.CNN_LSTM_Model(0.2, units=32,filters=32,kernel_size=1, normalization=False)
model.load_weights("saved_model\CNN_LSTM32_1_32_PosAssoluta")

In [20]:
model.summary()

Model: "CNN32_1LSTM32"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 30, 5)]           0         
                                                                 
 conv1d (Conv1D)             (None, 30, 32)            192       
                                                                 
 max_pooling1d (MaxPooling1D  (None, 30, 32)           0         
 )                                                               
                                                                 
 lstm_30 (LSTM)              (None, 32)                8320      
                                                                 
 dense_6 (Dense)             (None, 3)                 99        
                                                                 
Total params: 8,611
Trainable params: 8,611
Non-trainable params: 0
___________________________________________________

In [ ]:
import tf2onnx
import onnx
input_signature=(tf.TensorSpec([None,30,5],tf.float32,name='x'))
onnx_model,_=tf2onnx.converter.from_keras(model,input_signature,opset=9)